# GCP BigQueryのマニュアルを実行したログ

%load_ext マジック コマンドを使用して、BigQuery 用 IPython マジック コマンドを読み込みます。


- 実行日 2021/3/24

condaコマンドでインストールするとモジュールが古すぎでエラーが出る。そのため、pipコマンドのインストールじゃないと動かない。

以下を実行しておくこと
```
export GOOGLE_APPLICATION_CREDENTIALS=[json]
```

In [1]:
%load_ext google.cloud.bigquery

In [2]:
%%bigquery tax_forms --use_bqstorage_api
SELECT * FROM `bigquery-public-data.irs_990.irs_990_2012`

Downloading: 100%|██████████| 294019/294019 [00:53<00:00, 5526.34rows/s] 


In [3]:
%%bigquery stackoverflow --use_bqstorage_api
SELECT
  CONCAT(
    'https://stackoverflow.com/questions/',
    CAST(id as STRING)) as url,
  view_count
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags like '%google-bigquery%'
ORDER BY view_count DESC
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:03<00:00,  2.80rows/s]


In [4]:
import google.cloud.bigquery.magics

google.cloud.bigquery.magics.context.use_bqstorage_api = True

In [5]:
%%bigquery tax_forms
SELECT * FROM `bigquery-public-data.irs_990.irs_990_2012`

Downloading: 100%|██████████| 294019/294019 [00:29<00:00, 10073.71rows/s]


## Pythonクライアントライブラリを使用する。

In [6]:
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage

# Explicitly create a credentials object. This allows you to use the same
# credentials for both the BigQuery and BigQuery Storage clients, avoiding
# unnecessary API calls to fetch duplicate authentication tokens.
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Make clients.
bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [7]:
# Download query results.
query_string = """
SELECT
CONCAT(
    'https://stackoverflow.com/questions/',
    CAST(id as STRING)) as url,
view_count
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags like '%google-bigquery%'
ORDER BY view_count DESC
"""

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
print(dataframe.head())

                                            url  view_count
0  https://stackoverflow.com/questions/35159967       88467
1  https://stackoverflow.com/questions/10604135       84288
2  https://stackoverflow.com/questions/22879669       82252
3  https://stackoverflow.com/questions/27060396       65883
4  https://stackoverflow.com/questions/11057219       62127


In [8]:
# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.utility_us.country_code_iso"
)
rows = bqclient.list_rows(
    table,
    selected_fields=[
        bigquery.SchemaField("country_name", "STRING"),
        bigquery.SchemaField("fips_code", "STRING"),
    ],
)
dataframe = rows.to_dataframe(bqstorage_client=bqstorageclient)
print(dataframe.head())

       country_name fips_code
0          Akrotiri        AX
1   Bassas da India        BS
2          Dhekelia        DX
3     Europa Island        EU
4  Glorioso Islands        GO


In [10]:
#マニュアルにこの記述、書いていないような。
from google.cloud.bigquery_storage import types

In [11]:
project_id = "bigquery-public-data"
dataset_id = "new_york_trees"
table_id = "tree_species"
table = f"projects/{project_id}/datasets/{dataset_id}/tables/{table_id}"

# Select columns to read with read options. If no read options are
# specified, the whole table is read.
read_options = types.ReadSession.TableReadOptions(
    selected_fields=["species_common_name", "fall_color"]
)

parent = "projects/{}".format(your_project_id)

requested_session = types.ReadSession(
    table=table,
    # This API can also deliver data serialized in Apache Avro format.
    # This example leverages Apache Arrow.
    data_format=types.DataFormat.ARROW,
    read_options=read_options,
)
read_session = bqstorageclient.create_read_session(
    parent=parent, read_session=requested_session, max_stream_count=1,
)

# This example reads from only a single stream. Read from multiple streams
# to fetch data faster. Note that the session may not contain any streams
# if there are no rows to read.
stream = read_session.streams[0]
reader = bqstorageclient.read_rows(stream.name)

# Parse all Arrow blocks and create a dataframe. This call requires a
# session, because the session contains the schema for the row blocks.
dataframe = reader.to_dataframe(read_session)
print(dataframe.head())

  species_common_name fall_color
0         Shingle Oak     Yellow
1          Tulip Tree     Yellow
2    Northern Red Oak     Maroon
3          Coffeetree     Yellow
4            Sweetgum     Yellow


In [14]:
dataframe.shape

(57, 2)